In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from timm import create_model

# Перевіряємо, чи доступний GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. Ініціалізація моделі
model = create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 1)
model = model.to(device)

# 2. Завантажуємо state_dict
model.load_state_dict(torch.load('/content/best_model.pt', map_location=device), strict=False)
model.eval()

# 3. Підготовка зображення
image_path = '/content/smoker_data/Testing/Testing/smoking_0308.jpg'
image = Image.open(image_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_image = transform(image).unsqueeze(0).to(device)

# 4. Створюємо об'єкт Grad-CAM
target_layer = model.patch_embed.proj  # Використовуємо згортковий шар замість блоку трансформера

# 5. Створюємо Grad-CAM об'єкт
gradcam = GradCAM(model=model, target_layers=[target_layer])

# 6. Отримуємо CAM для зображення
grayscale_cam = gradcam(input_tensor=input_image)[0, :]  # Отримуємо теплову карту

# 7. Перетворюємо CAM на зображення та накладаємо його на оригінальне
# Нормалізуємо зображення для show_cam_on_image
rgb_image = np.array(image.resize((224, 224))) / 255.0  # Нормалізуємо до [0, 1]
cam_image = show_cam_on_image(rgb_image, grayscale_cam, use_rgb=True)

# 8. Показуємо зображення
plt.imshow(cam_image)
plt.axis('off')
plt.show()

# 9. Прогнозування класу
true_label = 1
with torch.no_grad():
    output = model(input_image)
    predicted_class = torch.sigmoid(output).item()
    print(f"Predicted: {predicted_class:.2f}, True label: {true_label}")